In [1]:
import torch
import pandas as pd
import numpy as np
import math
import random
from sklearn.model_selection import train_test_split

In [2]:
class BPR(object):
    def __init__(self):
        self.W = None             # user matrix
        self.W_np = None
        self.H = None             # item matrix
        self.H_np = None
        
        self.W_scnp = None
        self.H_scnp = None 
        self.rating_exp = {}
        self.rating_exp_mul_H = {}
        
        self.uid = None            # uid,iid without duplicates
        self.iid = None
        
        self.train_user_items = {}       # 用户u对应他访问过的所有items集合
        self.dev_user_items = {}
        
        self.uid_dict = None      # serialize uid and iid
        self.iid_dict = None      #  {(original id in dataset): (serial_idx)}
        self.uid_dict_rev = None  # reverse key and value
        self.iid_dict_rev = None  #  {(serial_idx): (original id in dataset)}
        
    def _split(self, df, ratio):
        train = pd.DataFrame(columns = df.columns, dtype=int)
        test = pd.DataFrame(columns = df.columns, dtype=int)
        for i in self.uid:
            train_1, test_1 = train_test_split(df[df.iloc[:, 0] == i], train_size = ratio)
            train = pd.concat([train, train_1])
            test = pd.concat([test, test_1])
        return train, test
    
    def split(self, df, train_size=0.8, test_size = 0.1):
        self.uid = np.asarray(list(set(df.iloc[:,0].values)))
        self.iid = np.asarray(list(set(df.iloc[:,1].values)))
        self.uid.sort()
        self.iid.sort()
        self.uid_dict = dict(zip(self.uid, [i for i in range(len(self.uid))]))
        self.iid_dict = dict(zip(self.iid, [i for i in range(len(self.iid))]))
        self.uid_dict_rev = {v : k for k, v in self.uid_dict.items()}
        self.iid_dict_rev = {v : k for k, v in self.iid_dict.items()}
        train, test = self._split(df, train_size)
        test, dev = self._split(test, test_size / (1 - train_size))
        return train, test, dev
    
    def generate_train_batch(self, batch, sets):
        train = []
        for b in range(batch):
            u = self.uid[random.randint(0, self.uid.size - 1)]
            i = sets[u][random.randint(0, sets[u].size - 1)]
            j = self.iid[random.randint(0, self.iid.size - 1)]
            while j in sets[u]:
                j = self.iid[random.randint(0, self.iid.size - 1)]
            train.append([self.uid_dict[u], self.iid_dict[i], self.iid_dict[j]])
        return np.asarray(train)   
        
    def scorer_prob(self, uids, iids, batch):              # Softmax probability
        prob = []                                          # uids, iids serial
        for b in range(batch):
            uid = uids[b].item()
            iid = iids[b].item()
            if uid not in self.rating_exp.keys():
                r = 0
                h = 0
                for i in self.train_user_items[self.uid_dict_rev[uid]]:
                    temp = np.exp(np.dot(self.W_scnp[uid], self.H_scnp[self.iid_dict[i]]))
                    r += temp
                    h += temp * self.H_scnp[self.iid_dict[i]]
                self.rating_exp[uid] = r
                self.rating_exp_mul_H[uid] = h
            prob.append(np.exp(np.dot(self.W_scnp[uid], self.H_scnp[iid])) / self.rating_exp[uid])
        prob = np.asarray(prob)
        return torch.from_numpy(prob).cuda()
    
    def fit(self, df, dev, k, max_iter=5, epoch=500, batch = 512):
        for u in self.uid:                                # 创建字典：用户u对应他访问过的所有items集合
            self.train_user_items[u] = df[df.iloc[:, 0]==u].iloc[:, 1].values
            self.dev_user_items[u] = dev[dev.iloc[:, 0]==u].iloc[:, 1].values
        # 初始化 W，H    
        self.W = torch.nn.Parameter(torch.rand(len(self.uid), k))      
        self.H = torch.nn.Parameter(torch.rand(len(self.iid), k))
        #初始化 scorer
        self.W_scnp = np.random.rand(len(self.uid), k)*0.01
        self.H_scnp = np.random.rand(len(self.iid), k)*0.01
        
        optimizer = torch.optim.Adam([self.W, self.H])
        for x in range(max_iter):             # Use stochastic batch gradient descent method to solve W & H
            loss = 0
            for e in range(epoch):
                #更新 W H
                uij = self.generate_train_batch(batch, self.train_user_items)
                u = torch.tensor(uij[:,0],dtype=torch.int32)
                i = torch.tensor(uij[:,1],dtype=torch.int32)
                j = torch.tensor(uij[:,2],dtype=torch.int32)
                u_emb = self.W[u].cuda()
                i_emb = self.H[i].cuda()
                j_emb = self.H[j].cuda()
                
                optimizer.zero_grad()
                score = self.scorer_prob(u, i, batch)
                
                normal_loss = -torch.mean(torch.log(torch.sigmoid(torch.sum(u_emb * (i_emb - j_emb), dim = 1))))
                normal_loss.backward()
                grad_W = self.W.grad.numpy()
                grad_H = self.H.grad.numpy()
                self.W.grad = None
                self.H.grad = None
                
                u_emb2 = self.W[u].cuda()
                i_emb2 = self.H[i].cuda()
                j_emb2 = self.H[j].cuda()
                bprloss = -torch.mean(score * torch.log(torch.sigmoid(torch.sum(u_emb2 * (i_emb2 - j_emb2), dim = 1))))
                bprloss.backward()
                optimizer.step()
                loss += bprloss 
                self.W.grad = None
                self.H.grad = None
                self.W_np = self.W.detach().numpy()
                self.H_np = self.H.detach().numpy()
                #更新 scorer 参数
                uij_dev = self.generate_train_batch(batch, self.dev_user_items)
                u_dev = torch.tensor(uij[:,0],dtype=torch.int32)
                i_dev = torch.tensor(uij[:,1],dtype=torch.int32)
                j_dev = torch.tensor(uij[:,2],dtype=torch.int32)
                u_dev_emb = self.W[u_dev].cuda()
                i_dev_emb = self.H[i_dev].cuda()
                j_dev_emb = self.H[j_dev].cuda()
                
                dev_loss = -torch.mean(torch.log(torch.sigmoid(torch.sum(u_dev_emb * (i_dev_emb - j_dev_emb), dim = 1))))
                dev_loss.backward()
                grad_W_on_dev = self.W.grad.numpy()
                grad_H_on_dev = self.H.grad.numpy()
                
                r_W = np.sum(grad_W * grad_W_on_dev, axis = 1)
                r_H = np.sum(grad_H * grad_H_on_dev, axis = 1)
                r_W = np.expand_dims(r_W, axis=0).repeat(k, axis = 0).T
                r_H = np.expand_dims(r_H, axis=0).repeat(k, axis = 0).T
                
                log_prob_W_grad = np.zeros((len(self.uid), k))
                log_prob_H_grad = np.zeros((len(self.iid), k))
                for b in range(batch):
                    u = uij[b,0]
                    i = uij[b,1]
                    log_prob_W_grad[u] = self.H_scnp[i] - self.rating_exp_mul_H[u] / self.rating_exp[u]
                    log_prob_H_grad[i] = self.W_scnp[u]*(1 - score[b].cpu().numpy())
                self.W_scnp += r_W * log_prob_W_grad
                self.H_scnp += r_H * log_prob_H_grad
 
                self.rating_exp = {}
                self.rating_exp_mul_H = {}
            print(f"Iteration: {x+1}, BPR loss: {loss/epoch}")   
        
#     def predict(self, user, n):      # Top-N recommendation
#         top_N = []
#         for i in self.iid:
#             if i not in self.train_user_items[user]:
#                 top_N.append((i, np.dot(self.W_np[self.uid_dict[user]], self.H_np[self.iid_dict[i]]))) 
#         return sorted(top_N, key=lambda s: s[1], reverse=True)[:n]
    
    def _predict(self, uid, items, n):
        top_N = []
        for i in range(len(items)):
            user = self.uid_dict[uid]
            item = self.iid_dict[items[i]]
            top_N.append((items[i], np.dot(self.W_np[user], self.H_np[item])))
        return sorted(top_N, key=lambda s: s[1], reverse=True)[:n]
    
    def NDCG(self, uid, test, n):         # 用模型排序+真实分数计算 DCG, 重排后计算 iDCG
        test_user = test[test.iloc[:, 0] == uid]
        rating = self._predict(uid, test_user.iloc[:, 1].values, n)
        irating =sorted(test_user.iloc[:, 2].values, reverse=True)
        dcg = 0
        idcg = 0
        if n > len(irating): n = len(irating)  
        for i in range(n):
            r = test_user[test_user.iloc[:, 1]==rating[i][0]].iloc[0, 2]
            dcg += 1.0*(2**r - 1)/math.log(i + 2, 2)
            idcg += 1.0*(2**irating[i] - 1)/math.log(i + 2, 2)
        return dcg/idcg
    
    def performance(self, test, n):      # Output recall@n, precision@n, NDCG@n
        hit = 0
        n_recall = 0
        n_precision = 0
        ndcg = 0
        for i in self.uid:
            # Items that User i hasn't tried in training set
            unknown_items = np.setdiff1d(self.iid, np.union1d(self.train_user_items[i], self.dev_user_items[i]))
            # Items that User i actually tried in testing set
            known_items = test[test.iloc[:, 0]==i].iloc[:, 1].values
            #目标：预测 unknown items 中的top_N，若击中test中的items，则为有效预测
            ru = self._predict(i, unknown_items, n)
            for item ,pui in ru:
                if item in known_items:
                    hit += 1
            n_recall += len(known_items)
            n_precision += n
            ndcg += self.NDCG(i, test, n)  
        recall = hit / (1.0 * n_recall)
        precision = hit / (1.0 * n_precision)
        ndcg /= len(self.uid)
        return recall, precision, ndcg

In [3]:
df1 = pd.read_csv("./ml-100k/u.data", sep="\t", names=['user id', 'item id', 'rating', 'timestamp'])
df2 = pd.read_csv("./ml-1m/ratings.dat", sep="::", names=['user id', 'item id', 'rating', 'timestamp'], engine='python')

In [4]:
model1 = BPR()
train1, test1, dev1 = model1.split(df1)
print(train1.shape)
print(test1.shape)
print(dev1.shape)

(79619, 4)
(9942, 4)
(10439, 4)


In [5]:
model1.fit(train1, dev1, 20)

Iteration: 1, BPR loss: 0.011047966614772448
Iteration: 2, BPR loss: 0.0065634269697210125
Iteration: 3, BPR loss: 0.005740098148399909
Iteration: 4, BPR loss: 0.00531856343244564
Iteration: 5, BPR loss: 0.005077416409057825


In [6]:
n = 10
rec, pre, ndcg = model1.performance(test1, n)
print(f'Precision@{n}: {pre}')
print(f'Recall@{n}: {rec}')
print(f'NDCG@{n}: {ndcg}')

Precision@10: 0.07507953340402969
Recall@10: 0.0712130356065178
NDCG@10: 0.8140134587711596


In [7]:
model2 = BPR()
train2, test2, dev2 = model2.split(df2)
print(train2.shape)
print(test2.shape)

(797758, 4)
(99692, 4)


In [8]:
model2.fit(train2, dev2, k = 20)

Iteration: 1, BPR loss: 0.010513865191826304
Iteration: 2, BPR loss: 0.006948783700702833
Iteration: 3, BPR loss: 0.0061021069422561545
Iteration: 4, BPR loss: 0.0057847897874899615
Iteration: 5, BPR loss: 0.005546645937397752


In [9]:
n = 10
rec, pre, ndcg = model2.performance(test2, n)
print(f'Precision@{n}: {pre}')
print(f'Recall@{n}: {rec}')
print(f'NDCG@{n}: {ndcg}')

Precision@10: 0.08932119205298013
Recall@10: 0.0541166793724672
NDCG@10: 0.8189502763082991
